In [2]:
import streamlit as st
from PyPDF2 import PdfFileReader
import streamlit_authenticator as stauth
# import json
from PIL import Image
import os
from dotenv import load_dotenv
from utils import AWSTexttract, LangChainAI, AWSS3, AWSTranscribe, DynamoDBManager
import yaml
from yaml.loader import SafeLoader
# from trubrics.integrations.streamlit import FeedbackCollector
import logging
load_dotenv('.env', override=True)
from mutagen.mp3 import MP3
from st_files_connection import FilesConnection
import datetime
from decimal import Decimal
from streamlit_cognito_auth import CognitoAuthenticator #https://github.com/pop-srw/streamlit-cognito-auth

In [2]:
text = ['ournee',
 'randi',
 'avero',
 'I più potenti booster di performance',
 'dello',
 'e',
 'ancora',
 'pass',
 'NON avevo distribuito quel compito su base giornalie-',
 'Tube che costem',
 'ra',
 '(vedere "I grandi progetti" nella prima parte); avevo',
 'O va al sempre. aveva',
 'invece aspettato il "momento opportuno" per dedicare',
 'alla scrittura delle giornate intere o delle mezze giornate',
 'etti',
 'giorno',
 'piu',
 'door',
 'energy',
 'Eppure suddividere un grande progetto in compiti più',
 'che',
 'piccoli da svolgere quotidianamente è alla base della mia',
 'So sul libro',
 'n continuation richiedecard',
 'filosofia di vita. Quel nuovo progetto aveva stravolto la',
 'mia solita logica organizzativa. Volevo scrivere con cura,',
 'riga per riga, parola per parola.',
 'giorni in CUI',
 'Come vi ho già raccontato, a quel punto decisi di cam-',
 'a. lo, che credevo non',
 'biare drasticamente metodo, adottando la tecnica del Na-',
 'NoWriMo: mi imposi quindi di scrivere cinquantamila pa-',
 'ssoluto per creare',
 'role in soli trenta giorni. Senza rileggere, senza giudicare',
 'o capace di spie',
 'né il contenuto né la forma. Poi mi restava ancora un mese',
 're, ho iniziato ao',
 'per ritoccare il testo, riscriverlo, limarlo, ma con la differen-',
 "za che lavoravo su qualcosa che già c'era. Così era molto",
 'O e, considerati',
 'più semplice ed efficace. Oltre a sentirmi particolarmente',
 'non mi si pote-',
 'fiero per aver caricato un video al giorno in un momento',
 'potuto trovare',
 'già piuttosto impegnativo, ho imparato anche una lezione:',
 'ettivamente la',
 'possiamo aggiungere al nostro planning un grande pro-',
 'getto con una scadenza lontana soltanto a tre condizioni',
 'ripetevo per',
 '. se ogni giorno riusciamo a dedicare qualche minuto al',
 'In questo',
 'progetto, con un obiettivo quotidiano quantificato;',
 'nella giorna-',
 'se ci prendiamo la libertà di buttare giù un primo abboz-',
 'zo, non definitivo, indipendentemente dal tipo di proget-',
 'berché non',
 'to è infatti più stimolante e semplice lavorare su un ma-',
 'titolo di un',
 'teriale grezzo già esistente che affannarsi sul "nulla".',
 'se ci imponiamo di non abbandonare il progetto per più',
 'di un giorno; e se per qualche ragione siamo costretti a',
 'dovuto',
 'prenderci una pausa più lunga. dobbiamo prima portarci',
 'libro sa-',
 'avanti con il lavoro: non dimenticate che una giornata',
 'ese. Mi',
 'persa è difficile da recuperare.',
 'cina.',
 '119',
 '']

### Create database

In [1]:
from utils import NotionAPI
import os

In [2]:
ROOT_PAGE="cdd4c4ceba3a45938f829d6fdcc52181"

In [3]:
notion = NotionAPI(os.getenv('NOTION_TOKEN'))
notion.creteDatabase(root=ROOT_PAGE, title='rio')

{"object":"error","status":400,"code":"validation_error","message":"body failed validation. Fix one:\nbody.properties.Name.number should be defined, instead was `undefined`.\nbody.properties.Name.formula should be defined, instead was `undefined`.\nbody.properties.Name.select should be defined, instead was `undefined`.\nbody.properties.Name.multi_select should be defined, instead was `undefined`.\nbody.properties.Name.status should be defined, instead was `undefined`.\nbody.properties.Name.relation should be defined, instead was `undefined`.\nbody.properties.Name.rollup should be defined, instead was `undefined`.\nbody.properties.Name.title should be an object, instead was `[{\"text\":{\"content\":\"rio\"}}]`.\nbody.properties.Name.rich_text should be defined, instead was `undefined`.\nbody.properties.Name.url should be defined, instead was `undefined`.\nbody.properties.Name.people should be defined, instead was `undefined`.\nbody.properties.Name.files should be defined, instead was `u

{'object': 'error',
 'status': 400,
 'code': 'validation_error',
 'message': 'body failed validation. Fix one:\nbody.properties.Name.number should be defined, instead was `undefined`.\nbody.properties.Name.formula should be defined, instead was `undefined`.\nbody.properties.Name.select should be defined, instead was `undefined`.\nbody.properties.Name.multi_select should be defined, instead was `undefined`.\nbody.properties.Name.status should be defined, instead was `undefined`.\nbody.properties.Name.relation should be defined, instead was `undefined`.\nbody.properties.Name.rollup should be defined, instead was `undefined`.\nbody.properties.Name.title should be an\xa0object, instead was `[{"text":{"content":"rio"}}]`.\nbody.properties.Name.rich_text should be defined, instead was `undefined`.\nbody.properties.Name.url should be defined, instead was `undefined`.\nbody.properties.Name.people should be defined, instead was `undefined`.\nbody.properties.Name.files should be defined, instead

### Check mp3 da S3

In [4]:
from mutagen.mp3 import MP3

In [5]:
audio = MP3(r"C:\Users\lafacero\Downloads\NPI-gagliano-2-pt2.mp3")
print(audio.info.length)

3595.2326530612245


### Estrazione testo da audio

Vedi [tutorial](https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a) 

In [3]:
from utils import NotionAPI, send_msg, AWSTexttract, LangChainAI, AWSS3, AWSTranscribe

In [5]:
transcribe = AWSTranscribe()
JOB_URI="s3://rioaws-transcribe-test/"
UPLOADED_MP3 = "NPI-gagliano-2-pt2.mp3"

job_name=transcribe.generate_job_name()
data = transcribe.amazon_transcribe(JOB_URI, job_name, UPLOADED_MP3, 'it-IT')

In [8]:
print(data)

press in termini allora mio l'effetto dello stress è questo c'è un asse che dalla così um l'ipotesi e poi la voce le ti lascio le cose e poi del del secondo un messaggero vabbè, sono il sistema, forse no. Vabbè, allora detto in generale c'è uno stimolo centrale che parte dall'ipod stimola la deposi che che concerne la c bianca, che è un ormone che arriva poi al surrene e fa produrre il cortisolo. Questa ma perde il cortisolo. A cosa serve? Serve a mettere il corpo nelle condizioni di rispondere alle condizioni di stress, perché bisogna che si vede figlio delle modifiche, non solo perché viene rilasciato contemporaneamente cade con lamine, cioè dopamina e dorata che servono a fare in modo che li divido. Prenda una decisione su come rispondere alla condizione minacciosa, stressante, ansiogena e in genere la risposta con scappi non ci sono molte alternative. Una delle due o la affronti la situazione stressante o la eviti. I vostri ragazzini opteranno. Per una delle due prevalentemente l'o

### Divisione testo (Text chunking)

The Langchain Character Text Splitter works by recursively dividing the text at specific characters.

It is especially useful for generic text.

By default it uses [“\n\n”, “\n”, “ ”, “”] as text separator

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# Initialize the text splitter with custom parameters
custom_text_splitter = RecursiveCharacterTextSplitter(
    # Set custom chunk size
    chunk_size = 100,
    chunk_overlap  = 20,
    # Use length of the text as the size measure, can also use separators = ['\n']
    length_function = len,
)

# Create the chunks
texts = custom_text_splitter.create_documents([data])

# Print the first two chunks
print(f'### Chunk 1: \n\n{texts[0].page_content}\n\n=====\n')
print(f'### Chunk 2: \n\n{texts[1].page_content}\n\n=====')

### Chunk 1: 

press in termini allora mio l'effetto dello stress è questo c'è un asse che dalla così um l'ipotesi

=====

### Chunk 2: 

così um l'ipotesi e poi la voce le ti lascio le cose e poi del del secondo un messaggero vabbè, sono

=====


In [31]:
texts[0]

Document(page_content="press in termini allora mio l'effetto dello stress è questo c'è un asse che dalla così um l'ipotesi", metadata={})

In [15]:
print(str(len(texts))+ " chunks")

479 chunks


By specifing the separator we obtain less and more long text chunk.

Remember, the ideal distribution depends on the specific requirements of your text-processing task. 

You might want smaller, more numerous chunks if you’re dealing with fine-grained analysis or larger, fewer chunks for broader semantic analysis.

##### Clustering Adjacent Sentences

The fundamental premise of this approach is that two sentences that appear consecutively in a text are more likely to be semantically related than two sentences that are farther apart.

It creates cluster of similar sentences.

In [18]:
import numpy as np
import spacy
nlp = spacy.load('it_core_news_sm')

In [19]:
# Load the Spacy model

def process(text):
    doc = nlp(text)
    sents = list(doc.sents)
    vecs = np.stack([sent.vector / sent.vector_norm for sent in sents])

    return sents, vecs

def cluster_text(sents, vecs, threshold):
    clusters = [[0]]
    for i in range(1, len(sents)):
        if np.dot(vecs[i], vecs[i-1]) < threshold:
            clusters.append([])
        clusters[-1].append(i)
    
    return clusters

def clean_text(text):
    # Add your text cleaning process here
    return text

In [20]:
# Initialize the clusters lengths list and final texts list
clusters_lens = []
final_texts = []

# Process the chunk
threshold = 0.3
sents, vecs = process(data)

# Cluster the sentences
clusters = cluster_text(sents, vecs, threshold)

for cluster in clusters:
    cluster_txt = clean_text(' '.join([sents[i].text for i in cluster]))
    cluster_len = len(cluster_txt)
    
    # Check if the cluster is too short
    if cluster_len < 60:
        continue
    
    # Check if the cluster is too long
    elif cluster_len > 3000:
        threshold = 0.6
        sents_div, vecs_div = process(cluster_txt)
        reclusters = cluster_text(sents_div, vecs_div, threshold)
        
        for subcluster in reclusters:
            div_txt = clean_text(' '.join([sents_div[i].text for i in subcluster]))
            div_len = len(div_txt)
            
            if div_len < 60 or div_len > 3000:
                continue
            
            clusters_lens.append(div_len)
            final_texts.append(div_txt)
            
    else:
        clusters_lens.append(cluster_len)
        final_texts.append(cluster_txt)

In [24]:
# Print the first two chunks
print(f'### Chunk 1: \n\n{final_texts[0]}\n\n=====\n')
print(f'### Chunk 2: \n\n{final_texts[1]}\n\n=====')

### Chunk 1: 

press in termini allora mio l'effetto dello stress è questo c'è un asse che dalla così um l'ipotesi e poi la voce le ti lascio le cose e poi del del secondo un messaggero vabbè, sono il sistema, forse no. Vabbè, allora detto in generale c'è uno stimolo centrale che parte dall'ipod stimola la deposi che che concerne la c bianca, che è un ormone che arriva poi al surrene e fa produrre il cortisolo. Questa ma perde il cortisolo. A cosa serve? Serve a mettere il corpo nelle condizioni di rispondere alle condizioni di stress, perché bisogna che si vede figlio delle modifiche, non solo perché viene rilasciato contemporaneamente cade con lamine, cioè dopamina e dorata che servono a fare in modo che li divido. Prenda una decisione su come rispondere alla condizione minacciosa, stressante, ansiogena e in genere la risposta con scappi non ci sono molte alternative. Una delle due o la affronti la situazione stressante o la eviti. I vostri ragazzini opteranno.

=====

### Chunk 2: 


In [36]:
len(max(final_texts, key = len))

2267

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=len(max(final_texts, key = len)), chunk_overlap=20)
docs = text_splitter.create_documents(final_texts)
docs

[Document(page_content="press in termini allora mio l'effetto dello stress è questo c'è un asse che dalla così um l'ipotesi e poi la voce le ti lascio le cose e poi del del secondo un messaggero vabbè, sono il sistema, forse no. Vabbè, allora detto in generale c'è uno stimolo centrale che parte dall'ipod stimola la deposi che che concerne la c bianca, che è un ormone che arriva poi al surrene e fa produrre il cortisolo. Questa ma perde il cortisolo. A cosa serve? Serve a mettere il corpo nelle condizioni di rispondere alle condizioni di stress, perché bisogna che si vede figlio delle modifiche, non solo perché viene rilasciato contemporaneamente cade con lamine, cioè dopamina e dorata che servono a fare in modo che li divido. Prenda una decisione su come rispondere alla condizione minacciosa, stressante, ansiogena e in genere la risposta con scappi non ci sono molte alternative. Una delle due o la affronti la situazione stressante o la eviti. I vostri ragazzini opteranno.", metadata={}

# Summarize

In [29]:
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI
langchain_client = LangChainAI()

In [39]:
llm = llm = OpenAI(temperature=0)
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose = True)  
summarized_data = chain.run(docs)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"press in termini allora mio l'effetto dello stress è questo c'è un asse che dalla così um l'ipotesi e poi la voce le ti lascio le cose e poi del del secondo un messaggero vabbè, sono il sistema, forse no. Vabbè, allora detto in generale c'è uno stimolo centrale che parte dall'ipod stimola la deposi che che concerne la c bianca, che è un ormone che arriva poi al surrene e fa produrre il cortisolo. Questa ma perde il cortisolo. A cosa serve? Serve a mettere il corpo nelle condizioni di rispondere alle condizioni di stress, perché bisogna che si vede figlio delle modifiche, non solo perché viene rilasciato contemporaneamente cade con lamine, cioè dopamina e dorata che servono a fare in modo che li divido. Prenda una decisione su come rispondere alla condizione minacciosa, stressante, ansiogena e in genere la risposta con scappi non ci sono molte alternative. Una delle du

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"

Lo stress è uno stimolo centrale che parte dall'ipod e stimola la produzione di ormoni come il cortisolo. Il cortisolo serve a mettere il corpo in condizioni di rispondere alle condizioni di stress. Quando si è sotto stress, le persone hanno due opzioni: affrontare la situazione o evitarla.

 Quando ci troviamo di fronte a un evento stressante, abbiamo la possibilità di scegliere tra due comportamenti: l'evitamento o la risposta interna. Se questi comportamenti vengono scelti in modo esasperato, possono diventare disturbi. A volte, quando siamo sotto stress, riusciamo a ricordare meglio le cose che abbiamo appena imparato.

 Quando siamo sotto stress, le nostre funzioni cognitive vengono potenziate, permettendoci di imparare più velocemente. Tuttavia, se lo stress si prolunga, questo effetto si inverte e può portare a una riduzione del

# Python timeout signals

See [tutorial](https://medium.com/@chamilad/timing-out-of-long-running-methods-in-python-818b3582eed6)

Note: this only works on Linux because Windows OS doesn't implement that signal.

In [6]:
import signal
import time
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [7]:
def timeout_handler(num, stack):
    '''
    We need to tell the signal module to execute our own function 
    when SIGALRM signal is received by the process. 
    So let’s first write a handler function.
    In the timeout_handler function we are raising an exception.
    '''
    logger.info("Received SIGALRM")
    raise Exception("SIGALRM")

In [8]:
def long_function():
    '''
    Change this to the long function that needs to be interrupted
    What this does is to simply wait for 60 seconds before continuing. 
    TThis is to emulate a blocked file read, or a hung server connection.
    '''
    time.sleep(60)
    return "Done"

In [9]:
#signal.alarm(10) tells the OS to send a SIGALRM after 10 seconds from this point onwards.
signal.signal(signal.SIGALRM, timeout_handler)
signal.alarm(10)

try:
    logger.info("Executing long function")
    long_function()
except Exception as ex:
    if "SIGALRM" in ex:
        logger.info("Timeout occurred")
    else:
        logger.info("Exception occurred")
finally:
    signal.alarm(0)
    logger.info("Alarm cleared")

AttributeError: module 'signal' has no attribute 'SIGALRM'

# With func_timeout

In [1]:
import time 
import func_timeout
import logging
logger = logging.getLogger(__name__)

ModuleNotFoundError: No module named 'func_timeout'

In [ ]:
def run_function(f, max_wait, default_value):
    try:
        start = time.time()
        x = func_timeout.func_timeout(max_wait, f)
        end = time.time()
        print("Function {} executed in {} seconds".format(f.__name__, end-start))
        return x
    except func_timeout.FunctionTimedOut:
        end = time.time()
        print("Function {} executed in {} seconds".format(f.__name__, end-start))
        pass
    return default_value

In [ ]:
run_function(long_function, 1, "timeout")

# DynamoDB

In [18]:
import boto3
import json

class DynamoDBManager:
    def __init__(self, region, table_name):
        self.region = region
        self.table_name = table_name
        self.dynamodb = boto3.resource('dynamodb', region_name=region)
        self.table = self.dynamodb.Table(table_name)
    
    def write_item(self, item):
        try:
            response = self.table.put_item(Item=item)
            print("Item added successfully:", response)
        except Exception as e:
            print("Error writing item:", e)
    
    def update_item(self, key, update_expression, expression_values):
        try:
            response = self.table.update_item(
                Key=key,
                UpdateExpression=update_expression,
                ExpressionAttributeValues=expression_values
            )
            print("Item updated successfully:", response)
        except Exception as e:
            print("Error updating item:", e)
    def get_item(self, key):
        try:
            response = self.table.get_item(Key=key)
            print("Item retrieved successfully:", response)
            return response
        except Exception as e:
            print("Error retrieving item:", e)

In [19]:
region_name = "eu-south-1"
table_name = "chatgpt-summary-users"

dynamo_manager = DynamoDBManager(region_name, table_name)

In [14]:
# Write a new item
new_item = {
    "email": "valore",
    "limits": "valore2"
}
dynamo_manager.write_item(new_item)

Item added successfully: {'ResponseMetadata': {'RequestId': 'GKQP8A3TIR1R6FQF0IQ748I1HNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 30 Aug 2023 12:24:19 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'GKQP8A3TIR1R6FQF0IQ748I1HNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [15]:
# Update an item
update_key = {
    "email": "valore"
}
update_expression = "SET limits = :new_value"
expression_values = {
    ":new_value": "nuovo_valore_2"
}
dynamo_manager.update_item(update_key, update_expression, expression_values)


Item updated successfully: {'ResponseMetadata': {'RequestId': 'GO2KLP3VPT39M3VQDPHLM036HBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 30 Aug 2023 12:25:16 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'GO2KLP3VPT39M3VQDPHLM036HBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [21]:
# Get an item
get_key = {
    "email": "valore"
}
res=dynamo_manager.get_item(get_key)
res['Item']['limits']

Item retrieved successfully: {'Item': {'email': 'valore', 'limits': 'nuovo_valore_2'}, 'ResponseMetadata': {'RequestId': 'HQBL3SBSP5AD60JVEHTGUKDJAJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 30 Aug 2023 12:27:12 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '65', 'connection': 'keep-alive', 'x-amzn-requestid': 'HQBL3SBSP5AD60JVEHTGUKDJAJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3860703557'}, 'RetryAttempts': 0}}


'nuovo_valore_2'

### Prompt fine tuning

In [1]:
#open from txt file
example_data=open('sample/sample_speech2text.txt', 'r').read()
example_data

"press in termini allora mio l'effetto dello stress Ã¨ questo c'Ã¨ un asse che dalla cosÃ¬ um l'ipotesi e poi la voce le ti lascio le cose e poi del del secondo un messaggero vabbÃ¨, sono il sistema, forse no. VabbÃ¨, allora detto in generale c'Ã¨ uno stimolo centrale che parte dall'ipod stimola la deposi che che concerne la c bianca, che Ã¨ un ormone che arriva poi al surrene e fa produrre il cortisolo. Questa \nma perde il cortisolo. A cosa serve? Serve a mettere il corpo nelle condizioni di rispondere alle condizioni di stress, perchÃ© bisogna che si vede figlio delle modifiche, non solo perchÃ© viene rilasciato \ncontemporaneamente cade con lamine, cioÃ¨ dopamina e dorata che servono a fare in modo che li divido. Prenda una decisione su come rispondere alla condizione minacciosa, stressante, ansiogena e in genere la risposta con scappi non ci sono molte alternative. Una delle due o la affronti la situazione stressante o la eviti. I vostri ragazzini opteranno. Per una delle due prev

In [28]:
from dotenv import load_dotenv
from utils import LangChainAI, TextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
load_dotenv()
langchain_client = LangChainAI()
llm = OpenAI(
          model_name="gpt-3.5-turbo-16k", # default model
          temperature=0.9)
text_summarizer = TextSplitter()

c:\Users\lafacero\venvs\streamlit\Lib\site-packages\langchain\llms\openai.py:158: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\lafacero\venvs\streamlit\Lib\site-packages\langchain\llms\openai.py:667: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [31]:
docs = text_summarizer.split_text(example_data) 
max_chunk_len = len(max(docs, key = len))
# max_chunk_len=1000
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=max_chunk_len, chunk_overlap=20)
docs = doc_splitter.create_documents(docs)

In [15]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
# Map
map_template = """Questi sono una serie di documenti
{docs}
Basandosi su questa lista di documenti, per favore restituisci una lista di bullet point che sintetizzano il contenuto dei documenti. 
"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)
print(map_chain.run(docs[0]))


• Il cortisolo è un ormone rilasciato dal sistema surrene in risposta allo stimolo ipotalamico
• Questa risposta viene attivata per affrontare lo stress
• Il cortisolo raggiunge la deposi bianca e stimola la produzione di ormoni
• L'effetto dello stress può diminuire la produzione di cortisolo


In [33]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = """Scrivi una lista di bullet point che sintetizzano il contenuto dei documenti.:
"{text}"
Lista dei bullet points:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

print(stuff_chain.run(docs))

- Lo stress può avere effetti negativi sulla salute mentale e fisica
- Lo stress attiva la produzione di cortisolo, un ormone che mette il corpo in uno stato di allerta per affrontare le situazioni stressanti
- Le risposte allo stress possono variare tra affrontamento attivo o evitamento
- L'eccessivo evitamento cronico può portare a disturbi del comportamento
- Lo stress prolungato può interferire con la neurogenesi e la plasticità cerebrale
- Durante periodi di stress, si può sperimentare un potenziamento della memoria e delle funzioni cognitive
- Tuttavia, lo stress cronico può portare a danni neuronal e riduzione della memoria
- I bambini esposti a esperienze avverse durante l'infanzia sono più a rischio di sviluppare problemi di salute mentale e difficoltà cognitive in futuro
- La pandemia e le misure di contenimento adottate hanno avuto un impatto negativo sul neurosviluppo e sulla salute mentale dei bambini e degli adolescenti
- L'uso di sostanze d'abuso, come alcol e droghe, è 

### Test e2e

In [1]:
import streamlit as st
from PyPDF2 import PdfFileReader
import streamlit_authenticator as stauth
# import json
from PIL import Image
import os
from dotenv import load_dotenv
from utils import AWSTexttract, LangChainAI, AWSS3, AWSTranscribe, DynamoDBManager
import yaml
from yaml.loader import SafeLoader
# from trubrics.integrations.streamlit import FeedbackCollector
import logging
load_dotenv('.env', override=True)
from mutagen.mp3 import MP3
from st_files_connection import FilesConnection
import datetime
from decimal import Decimal
from streamlit_cognito_auth import CognitoAuthenticator #https://github.com/pop-srw/streamlit-cognito-auth
# import subprocess

In [2]:
lang="ITA"
JOB_URI="s3://riassume-transcribe-bucket/"
S3_BUCKET='riassume-transcribe-bucket'
COGNITO_USER_POOL='us-east-1_2gJgqtGK3'
COGNITO_CLIENT_ID='1hbdf29bl3goifqovdsga02kov'
table_name = "chatgpt-summary-users"
langchain_client = LangChainAI()
s3_client=AWSS3('riassume-document-bucket')
conn = st.experimental_connection('s3', type=FilesConnection)
transcribe_s3client = AWSS3(S3_BUCKET)
transcribe = AWSTranscribe(JOB_URI)
textract = AWSTexttract()
dynamo_manager = DynamoDBManager(os.getenv('AWS_REGION'), table_name)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
# UPLOAD_FOLDER = '/tmp' #on Linux/Docker
UPLOAD_FOLDER = r"C:\Users\ELAFACRB1\Codice\GitHub\chatgpt-summmary\uploads" #on Winzozz
SQS_URL = os.getenv('SQS_URL')

2023-11-28 15:04:04.853 
  command:

    streamlit run c:\Users\ELAFACRB1\Codice\GitHub\chatgpt-summmary\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
with open('test\script_estratto.txt', 'r') as file:
    example_data = file.read().replace('\n', '')

In [4]:
username='ros'
audio_filename='NPI-gagliano-2-pt2.mp3'

In [5]:
get_key = {"username": username}
mins_left=dynamo_manager.get_item(get_key)['Item']['time_limit']
print('mins left: ', mins_left)
if mins_left<0:
    st.error("Non hai più minuti disponibili per la versione di prova! Contattaci per continuare ad usare l'applicazione.")
    raise Exception("User out of time!")

Item retrieved successfully: {'Item': {'time_limit': Decimal('3604.767346938775517628528178'), 'n_images': Decimal('2'), 'username': 'ros'}, 'ResponseMetadata': {'RequestId': '5I1OQ5DTSLDVPKV262CR06VSRFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Tue, 28 Nov 2023 14:04:04 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '105', 'connection': 'keep-alive', 'x-amzn-requestid': '5I1OQ5DTSLDVPKV262CR06VSRFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2439645444'}, 'RetryAttempts': 0}}
mins left:  3604.767346938775517628528178


In [6]:
def speech_to_text(job_name, audio_filename, lang_code):
    job_name=transcribe.generate_job_name()
    data = transcribe.amazon_transcribe(JOB_URI, job_name, audio_filename, lang_code)
    logger.info("File audio transcribed!")
    return data

In [7]:
summarized_data = langchain_client.summarize_text(example_data)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"poi la stabilità. Giulia ha una buona adattabilità e questo è un tratto estremamente produttivo. Incontrerete in genere questo tratto è abbastanza basso in termini di possibilità di adattarsi al contesto."


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"Ci sono bambini molto adattabili che hanno patologie psichiatriche che hanno problemi. Però però mediamente questo tratto è un tratto che fa rischiare il disturbo psicopatologico, perché meno sia adattabile, più gli accidenti della vita che ti stressano ti segnano. E poi l'approccio ritiro, vado verso l'approccio, allora questo è un tratto fondamentale. Vi ricordate approccio ritiro venivate definito."


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"E quindi Giulia, tra l'esperienza nuova e l'esperienza 

In [8]:
summarized_data

"Il brano discute una vasta gamma di argomenti, tra cui adattabilità, disturbi psichiatrici, ripetizione nell'apprendimento, esperienze sensoriali, motivazione, ansia, trauma, terapia, narrazione, strategie COVID-19, attaccamento, disturbi comportamentali, diagnosi di trauma, disturbi mentali e dipendenza, dissociazione, depersonalizzazione, ansia nei bambini, terapia dell'autismo, esplorazione degli ambienti per persone disabili e l'importanza di informare gli altri sulle proprie prospettive personali. Sottolinea l'importanza di comprendere i propri processi di pensiero e le esperienze passate nel plasmare il comportamento e le risposte emotive.\n\n"

In [8]:
summarized_data="Il brano discute una vasta gamma di argomenti, tra cui adattabilità, disturbi psichiatrici, ripetizione nell'apprendimento, esperienze sensoriali, motivazione, ansia, trauma, terapia, narrazione, strategie COVID-19, attaccamento, disturbi comportamentali, diagnosi di trauma, disturbi mentali e dipendenza, dissociazione, depersonalizzazione, ansia nei bambini, terapia dell'autismo, esplorazione degli ambienti per persone disabili e l'importanza di informare gli altri sulle proprie prospettive personali. Sottolinea l'importanza di comprendere i propri processi di pensiero e le esperienze passate nel plasmare il comportamento e le risposte emotive.\n\n"

bullet points

In [9]:
bullet_point_text = langchain_client.bullet_point_text(summarized_data)
bullet_point_text

"- Argomenti: adattabilità, disturbi psichiatrici, ripetizione nell'apprendimento, esperienze sensoriali, motivazione, ansia, trauma, terapia, narrazione, strategie COVID-19, attaccamento, disturbi comportamentali, diagnosi di trauma, disturbi mentali e dipendenza, dissociazione, depersonalizzazione, ansia nei bambini, terapia dell'autismo, esplorazione degli ambienti per persone disabili, importanza di informare sulle prospettive personali.\n- Importanza di comprendere i propri processi di pensiero.\n- Esperienze passate influenzano il comportamento e le risposte emotive."